In [1]:
#install.packages("magrittr")
#install.packages('purrr')
#install.packages('glue')
#install.packages('stringr')
#install.packages('rvest')
#install.packages('xml2')
#install.packages('tidyverse')
library(tidyverse)
library(magrittr) # better handling of pipes
library(purrr) # to work with lists and map functions
library(glue) # to paste strings
library(stringr) # to hand strings
library(rvest) # rvest makes scraping easier
library(xml2) # makes it easier to work with HTML and XML from R
library(tidyr)
library(fuzzyjoin)

-- Attaching packages --------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.6     v purrr   0.3.4
v tibble  3.1.8     v dplyr   1.0.9
v tidyr   1.2.0     v stringr 1.4.1
v readr   2.1.2     v forcats 0.5.1

-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


Attaching package: 'magrittr'


The following object is masked from 'package:purrr':

    set_names


The following object is masked from 'package:tidyr':

    extract



Attaching package: 'rvest'


The following object is masked from 'package:readr':

    guess_encoding




In [2]:
#Read in the migration datafile
migration <- read_csv(file = "internal_migration.csv", show_col_types = FALSE) 

head(migration)

year_ended,age_grp,sex,origin_talb17,origin_talb17desc,destination_talb17,destination_talb17desc,count
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
30/06/2014,0-4,Female,1,Far North district,1,Far North district,477
30/06/2014,0-4,Female,1,Far North district,2,Whangarei district,51
30/06/2014,0-4,Female,1,Far North district,3,Kaipara district,6
30/06/2014,0-4,Female,1,Far North district,13,Waikato district,9
30/06/2014,0-4,Female,1,Far North district,15,Matamata-Piako district,3
30/06/2014,0-4,Female,1,Far North district,16,Hamilton city,9


In [3]:
#Show the unique entries in the migratin dataset
newlist <- unique(migration$destination_talb17desc)
"Far North district" %in% newlist

[1] TRUE

In [4]:
#Change the dataframe to identify the year
migration <- migration %>%
  separate(year_ended, sep="/", into = c("day","month","year"))



In [5]:
council_URL = "https://www.aucklandcouncil.govt.nz/about-auckland-council/how-auckland-council-works/local-boards/all-local-boards/Pages/default.aspx"
council <- read_html(council_URL)

boards <- council_URL %>%
  read_html() %>%
  html_nodes(".page-link-with-description-title") %>%
  html_text()

boards

[1] "Albert-Eden Local Board"           "Aotea / Great Barrier Local Board"
 [3] "Devonport-Takapuna Local Board"    "Franklin Local Board"             
 [5] "Henderson-Massey Local Board"      "Hibiscus and Bays Local Board"    
 [7] "Howick Local Board"                "Kaip<U+0101>tiki Local Board"     
 [9] "M<U+0101>ngere-<U+014C>t<U+0101>huhu Local Board" "Manurewa Local Board"             
[11] "Maungakiekie-T<U+0101>maki Local Board" "<U+014C>r<U+0101>kei Local Board" 
[13] "<U+014C>tara-Papatoetoe Local Board" "Papakura Local Board"             
[15] "Puket<U+0101>papa Local Board"     "Rodney Local Board"               
[17] "Upper Harbour Local Board"         "Waiheke Local Board"              
[19] "Wait<U+0101>kere Ranges Local Board" "Waitemat<U+0101> Local Board"     
[21] "Whau Local Board"

In [6]:
#Import the TA region

TA_URL <- "https://en.wikipedia.org/wiki/Territorial_authorities_of_New_Zealand"

TA <- read_html(TA_URL)
TA_tables <- html_table(TA)
TA_table<-TA_tables[2]

scraped_info <- tibble(Name=TA_table[[1]][[1]], Region = TA_table[[1]][[7]])

#Scrape for the Auckland Council regions

council_URL = "https://www.aucklandcouncil.govt.nz/about-auckland-council/how-auckland-council-works/local-boards/all-local-boards/Pages/default.aspx"
council <- read_html(council_URL)

boards <- council_URL %>%
  read_html() %>%
  html_nodes(".page-link-with-description-title") %>%
  html_text()

#Split the Region variable with spaces,
final <- scraped_info %>%
  mutate(Region1 = stringr::str_split(Region, " ") %>% map_chr(., 1)) %>%
  as.data.frame()

final <- final %>%
add_row(Name=boards, Region = "Auckland", Region1 = "Auckland")


final <- as_tibble(final)


#Edit the areas with unitary authority

final %>%
filter(Region1 == "unitary")

Name,Region,Region1
<chr>,<chr>,<chr>
Auckland Council,unitary authority,unitary
Gisborne District Council,unitary authority,unitary
Tasman District Council,unitary authority,unitary
Nelson City Council,unitary authority,unitary
Marlborough District Council,unitary authority,unitary
Chatham Islands District Council,unitary authority,unitary


In [7]:
final<-subset(final, Name!= "Auckland Council")
final<-subset(final, Name!= "Chatham Islands District Council") 
final[final$Name=="Gisborne District Council",]$Region1 <- "Gisborne"
final[final$Name=="Marlborough District Council",]$Region1 <- "Marlborough"
final[final$Name=="Tasman District Council",]$Region1 <- "Tasman"
final[final$Name=="Nelson City Council",]$Region1 <- "Nelson"

In [8]:
final %>% 
filter(Region1 == "unitary")

final

Name,Region,Region1
<chr>,<chr>,<chr>


Name,Region,Region1
<chr>,<chr>,<chr>
Far North District Council,Northland,Northland
Whang<U+0101>rei District Council,Northland,Northland
Kaipara District Council,Northland,Northland
Thames-Coromandel District Council,Waikato,Waikato
Hauraki District Council,Waikato,Waikato
Waikato District Council,Waikato,Waikato
Matamata-Piako District Council,Waikato,Waikato
Hamilton City Council,Waikato,Waikato
Waipa District Council,Waikato,Waikato


In [9]:
head(migration)

day,month,year,age_grp,sex,origin_talb17,origin_talb17desc,destination_talb17,destination_talb17desc,count
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
30,06,2014,0-4,Female,1,Far North district,1,Far North district,477
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9
30,06,2014,0-4,Female,1,Far North district,15,Matamata-Piako district,3
30,06,2014,0-4,Female,1,Far North district,16,Hamilton city,9


In [10]:
final<- final %>% rename(origin_talb17desc=Name)
head(final)

head(migration)

origin_talb17desc,Region,Region1
<chr>,<chr>,<chr>
Far North District Council,Northland,Northland
Whang<U+0101>rei District Council,Northland,Northland
Kaipara District Council,Northland,Northland
Thames-Coromandel District Council,Waikato,Waikato
Hauraki District Council,Waikato,Waikato
Waikato District Council,Waikato,Waikato


day,month,year,age_grp,sex,origin_talb17,origin_talb17desc,destination_talb17,destination_talb17desc,count
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>
30,06,2014,0-4,Female,1,Far North district,1,Far North district,477
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9
30,06,2014,0-4,Female,1,Far North district,15,Matamata-Piako district,3
30,06,2014,0-4,Female,1,Far North district,16,Hamilton city,9


In [11]:
#Add a departing region to the migration dataset

migration <- migration %>%
stringdist_left_join(final, method = 'lcs', by = "origin_talb17desc", max_dist=15)
migration <- migration %>% rename(OriginRegion=Region1)


head(migration)

day,month,year,age_grp,sex,origin_talb17,origin_talb17desc.x,destination_talb17,destination_talb17desc,count,origin_talb17desc.y,Region,OriginRegion
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>
30,06,2014,0-4,Female,1,Far North district,1,Far North district,477,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,15,Matamata-Piako district,3,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,16,Hamilton city,9,Far North District Council,Northland,Northland


In [12]:
final <- final %>% rename(destination_talb17desc=origin_talb17desc)
head(final)

destination_talb17desc,Region,Region1
<chr>,<chr>,<chr>
Far North District Council,Northland,Northland
Whang<U+0101>rei District Council,Northland,Northland
Kaipara District Council,Northland,Northland
Thames-Coromandel District Council,Waikato,Waikato
Hauraki District Council,Waikato,Waikato
Waikato District Council,Waikato,Waikato


In [13]:


migration <- migration %>%
stringdist_left_join(final, method = 'lcs', by = "destination_talb17desc", max_dist=15)


migration <- migration %>% rename(DestinationRegion=Region1)


head(migration)

day,month,year,age_grp,sex,origin_talb17,origin_talb17desc.x,destination_talb17,destination_talb17desc.x,count,origin_talb17desc.y,Region.x,OriginRegion,destination_talb17desc.y,Region.y,DestinationRegion
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
30,06,2014,0-4,Female,1,Far North district,1,Far North district,477,Far North District Council,Northland,Northland,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51,Far North District Council,Northland,Northland,Whang<U+0101>rei District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51,Far North District Council,Northland,Northland,Whanganui District Council,Manawat<U+016B>-Whanganui,Manawat<U+016B>-Whanganui
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Kaipara District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Waipa District Council,Waikato,Waikato
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Wairoa District Council,Hawke's Bay,Hawke's


In [14]:
migration

day,month,year,age_grp,sex,origin_talb17,origin_talb17desc.x,destination_talb17,destination_talb17desc.x,count,origin_talb17desc.y,Region.x,OriginRegion,destination_talb17desc.y,Region.y,DestinationRegion
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
30,06,2014,0-4,Female,1,Far North district,1,Far North district,477,Far North District Council,Northland,Northland,Far North District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51,Far North District Council,Northland,Northland,Whang<U+0101>rei District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51,Far North District Council,Northland,Northland,Whanganui District Council,Manawat<U+016B>-Whanganui,Manawat<U+016B>-Whanganui
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Kaipara District Council,Northland,Northland
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Waipa District Council,Waikato,Waikato
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Wairoa District Council,Hawke's Bay,Hawke's
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Kaik<U+014D>ura District Council,Canterbury,Canterbury
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9,Far North District Council,Northland,Northland,Waikato District Council,Waikato,Waikato
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9,Far North District Council,Northland,Northland,Waipa District Council,Waikato,Waikato


In [17]:
head(migration)

day,month,year,age_grp,sex,origin_talb17,origin_talb17desc.x,destination_talb17,destination_talb17desc.x,count,origin_talb17desc.y,Region.x,OriginRegion,destination_talb17desc.y,Region.y,DestinationRegion
<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
30,06,2014,0-4,Female,1,Far North district,2,Whangarei district,51,Far North District Council,Northland,Northland,Whanganui District Council,Manawat<U+016B>-Whanganui,Manawat<U+016B>-Whanganui
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Waipa District Council,Waikato,Waikato
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Wairoa District Council,Hawke's Bay,Hawke's
30,06,2014,0-4,Female,1,Far North district,3,Kaipara district,6,Far North District Council,Northland,Northland,Kaik<U+014D>ura District Council,Canterbury,Canterbury
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9,Far North District Council,Northland,Northland,Waikato District Council,Waikato,Waikato
30,06,2014,0-4,Female,1,Far North district,13,Waikato district,9,Far North District Council,Northland,Northland,Waipa District Council,Waikato,Waikato


In [15]:
#Get the values for people entering each district
migration <- migration %>%
 filter(migration$OriginRegion!=migration$DestinationRegion)

entering_region <- aggregate(migration$count, by=list(Category = migration$year, migration$DestinationRegion), FUN=sum)



#Get the values for the number of people leaving the district
departing_region <- aggregate(migration$count, by=list(Category = migration$year, migration$OriginRegion), FUN=sum  )

head(entering_region)

,Category,Group.2,x
,<chr>,<chr>,<dbl>
1,2014,Auckland,49401
2,2015,Auckland,52344
3,2016,Auckland,53199
4,2017,Auckland,50583
5,2014,Bay,20613
6,2015,Bay,23820


In [18]:
"#Get the values for people entering each district
entering_district <- aggregate(migration$count, by=list(Category = migration$year, migration$destination_talb17desc.y), FUN=sum)

entering_district <- entering_district%>%
  rename(Name=Group.2)
head(entering_district)

#Get the values for the number of people leaving the district
departing_district <- aggregate(migration$count, by=list(Category = migration$year, migration$origin_talb17desc.x), FUN=sum  )
departing_district <- departing_district%>%
  rename(Name=Group.2)
head(departing_district)"

,Category,Name,x
,<chr>,<chr>,<dbl>
1,2014,Albert-Eden Local Board,3039
2,2015,Albert-Eden Local Board,3132
3,2016,Albert-Eden Local Board,2838
4,2017,Albert-Eden Local Board,2784
5,2014,Ashburton District Council,1725
6,2015,Ashburton District Council,1848


,Category,Name,x
,<chr>,<chr>,<dbl>
1,2014,Albert-Eden local board area,2853
2,2015,Albert-Eden local board area,3114
3,2016,Albert-Eden local board area,3060
4,2017,Albert-Eden local board area,3381
5,2014,Ashburton district,1707
6,2015,Ashburton district,1815


In [22]:
#make a table showing the net migration for each district

joined_regional <- merge(x=entering_region, y=departing_region,by=c("Group.2","Category")) %>%
rename(Departing=x.x, Entering=x.y, Region = Group.2)



joined_regional <- joined_regional %>%
  mutate(netmigration = Entering-Departing) 

joined_regional

Region,Category,Departing,Entering,netmigration
<chr>,<chr>,<dbl>,<dbl>,<dbl>
Auckland,2014,49401,54831,5430
Auckland,2015,52344,65733,13389
Auckland,2016,53199,72780,19581
Auckland,2017,50583,74862,24279
Bay,2014,20613,19689,-924
Bay,2015,23820,22446,-1374
Bay,2016,25836,23298,-2538
Bay,2017,24840,23259,-1581
Canterbury,2014,145224,145152,-72


In [ ]:
#test <- merge(x=departing_district, y= final, by=startswith("Group.2"))
departing_regions <- departing_district %>%
stringdist_left_join(final, method = 'lcs', by="Name", max_dist = 15)

test

In [ ]:
arriving_regions <- entering_district %>%
stringdist_left_join(final, method = 'lcs', by = "Name", max_dist=15)

arriving_regions <- subset(arriving_regions, select = -c(Region, Name.x))

head(arriving_regions)
